# IF3170 Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 42

Group Members:
- Shazya Audrea Taufik (13522063)
- Ahmad Mudabbir Arif (13522072)
- Zahira Dina Amalia (13522085)
- Naufal Adnan (13522116)

## Import Libraries

In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

import pickle
import concurrent.futures
from os import cpu_count
from tqdm import tqdm
import time
from collections import Counter

## Constants

In [18]:
TARGET = "attack_cat"
RANDOM_STATE = 42

NUMERICALFEATURES = ['ackdat', 'ct_dst_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'ct_flw_http_mthd', 'ct_ftp_cmd', 'ct_src_dport_ltm', 'ct_src_ltm', 'ct_srv_dst', 'ct_srv_src', 'ct_state_ttl', 'dbytes', 'dinpkt', 'djit', 'dload', 'dloss', 'dmean', 'dpkts', 'dtcpb', 'dttl', 'dur', 'dwin', 'response_body_len', 'sbytes', 'sinpkt', 'sjit', 'sload', 'sloss', 'smean', 'spkts', 'stcpb', 'sttl', 'swin', 'synack', 'tcprtt', 'trans_depth']
CATEGORICALFEATURES = ['is_ftp_login', 'is_sm_ips_ports', 'proto', 'service', 'state']

## Import Dataset

In [19]:
additional_train = pd.read_csv('https://drive.google.com/uc?id=1JmsE7idHRmzqVnWuUQhCopu3CWcVuE4D')
basic_train = pd.read_csv('https://drive.google.com/uc?id=1Cq0dGTBTUAni612F4eV_HFCZKCxh1_aK')
content_train = pd.read_csv('https://drive.google.com/uc?id=1kdtMn-rdfvELI49GCIM-7wKxJqQpHmj8')
flow_train = pd.read_csv('https://drive.google.com/uc?id=1OWFC-fxP--Ez2T7J_O11nJOAUb3qpINL')
time_train = pd.read_csv('https://drive.google.com/uc?id=1dFH5ejQ6CxSpJ01ppJ_juvI16KzMWKYV')
labels_train = pd.read_csv('https://drive.google.com/uc?id=1gavJNwHYRWZ2gXn9r2vE6rsC2qeNbWWd')

train_data = (additional_train
              .merge(basic_train, on='id', how='inner')
              .merge(content_train, on='id', how='inner')
              .merge(flow_train, on='id', how='inner')
              .merge(time_train, on='id', how='inner')
              .merge(labels_train, on='id', how='inner'))

train_data

,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,...,proto,sjit,djit,sinpkt,dinpkt,tcprtt,synack,ackdat,attack_cat,label
0,0.0,0.0,0.0,0.0,0.0,11.0,NaN,5.0,4.0,2.0,...,tcp,4449.110313,3234.831566,11.845558,6.261361,NaN,0.000444,0.000114,Normal,0
1,0.0,2.0,0.0,0.0,0.0,10.0,10.0,10.0,10.0,NaN,...,udp,0.000000,0.000000,0.009000,0.000000,0.000000,0.000000,NaN,Generic,1
2,0.0,NaN,0.0,0.0,0.0,4.0,4.0,2.0,2.0,1.0,...,tcp,8561.040438,249.950547,165.386453,172.345750,0.158826,0.057902,0.100924,Exploits,1
3,0.0,0.0,0.0,0.0,0.0,9.0,9.0,3.0,2.0,2.0,...,tcp,4053.086020,2918.730804,8.669644,4.496707,0.000558,0.000448,NaN,Normal,0
4,0.0,0.0,0.0,0.0,0.0,3.0,3.0,4.0,3.0,1.0,...,udp,0.000000,0.000000,0.008000,0.007000,0.000000,0.000000,0.000000,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,0.0,2.0,0.0,0.0,0.0,38.0,38.0,32.0,NaN,32.0,...,udp,0.000000,0.000000,0.005000,0.000000,0.000000,0.000000,0.000000,Generic,1
175337,0.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,3.0,NaN,...,tcp,15464.286700,531.848125,193.445428,209.600766,0.066524,0.047738,0.018786,Exploits,1
175338,0.0,2.0,0.0,NaN,0.0,33.0,33.0,16.0,16.0,16.0,...,udp,0.000000,0.000000,0.008000,0.000000,0.000000,0.000000,0.000000,Generic,1
175339,0.0,2.0,0.0,0.0,0.0,36.0,36.0,NaN,37.0,36.0,...,udp,0.000000,0.000000,0.004000,0.000000,0.000000,0.000000,NaN,Generic,1


In [20]:
additional_test = pd.read_csv('https://drive.google.com/uc?id=17lMNbUqPi6Qb5aCgPLCDNb3OOVTVfzi1')
basic_test = pd.read_csv('https://drive.google.com/uc?id=1z2U40sV7TjgPyxNq7MDPT_3MSNWX16ae')
content_test = pd.read_csv('https://drive.google.com/uc?id=1IzHNN_QnKEwoJFcQNmMK0hCVNJ3LgNyx')
flow_test = pd.read_csv('https://drive.google.com/uc?id=1JHTe6skur9mt5uwofduMMCRXCQieqWKa')
time_test = pd.read_csv('https://drive.google.com/uc?id=1KKxW42gIm5tfgYL4gULt4aI7-dOADmTI')

test_data = (additional_test
             .merge(basic_test, on='id', how='inner')
             .merge(content_test, on='id', how='inner')
             .merge(flow_test, on='id', how='inner')
             .merge(time_test, on='id', how='inner'))

test_data

,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,...,trans_depth,response_body_len,proto,sjit,djit,sinpkt,dinpkt,tcprtt,synack,ackdat
0,0.0,1.0,0.0,0.0,0.0,5.0,5.0,2.0,2.0,2.0,...,0.0,0.0,tcp,2737.954123,118.833969,48.756556,76.593602,0.165117,0.072001,0.093116
1,0.0,1.0,0.0,NaN,0.0,6.0,6.0,1.0,1.0,1.0,...,0.0,0.0,tcp,2938.299144,165.780563,49.812539,109.557602,0.223604,0.100248,0.123356
2,0.0,1.0,0.0,0.0,0.0,4.0,4.0,1.0,2.0,1.0,...,0.0,0.0,tcp,4287.453629,129.471406,69.765530,94.395906,0.113189,0.082498,0.030691
3,0.0,2.0,0.0,0.0,0.0,10.0,4.0,2.0,4.0,2.0,...,0.0,0.0,udp,0.000000,0.000000,0.001000,0.000000,0.000000,0.000000,0.000000
4,NaN,0.0,NaN,0.0,0.0,13.0,11.0,10.0,7.0,6.0,...,0.0,0.0,tcp,1119.063538,26.748141,17.628799,15.543294,0.000655,0.000526,0.000129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20578,0.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,...,0.0,0.0,tcp,1902.551328,143.722203,16.088482,87.730055,0.117474,0.071119,0.046355
20579,0.0,0.0,0.0,0.0,0.0,14.0,7.0,1.0,9.0,1.0,...,0.0,0.0,tcp,858.139465,22.118752,14.832222,13.803000,0.000647,0.000491,0.000156
20580,0.0,2.0,0.0,0.0,0.0,6.0,6.0,NaN,4.0,4.0,...,0.0,0.0,udp,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000
20581,0.0,2.0,0.0,0.0,0.0,11.0,11.0,4.0,8.0,4.0,...,0.0,0.0,unas,0.000000,0.000000,0.011000,0.000000,0.000000,0.000000,0.000000


# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [ ]:
# Write your code here

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [21]:
train_set, val_set = train_test_split(train_data, test_size=0.2, random_state=42)

In [22]:
train_set

,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,...,proto,sjit,djit,sinpkt,dinpkt,tcprtt,synack,ackdat,attack_cat,label
96203,0.0,1.0,0.0,0.0,0.0,5.0,5.0,1.0,1.0,1.0,...,NaN,4604.553147,102.742672,86.430667,81.175219,0.122210,0.048200,0.074010,Normal,0
58960,0.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,1.0,...,mhrp,0.000000,0.000000,0.009000,0.000000,0.000000,0.000000,0.000000,Exploits,1
65069,0.0,2.0,0.0,0.0,0.0,28.0,28.0,17.0,17.0,17.0,...,udp,0.000000,0.000000,0.009000,0.000000,0.000000,0.000000,0.000000,Generic,1
64133,0.0,2.0,0.0,0.0,0.0,10.0,10.0,10.0,10.0,10.0,...,udp,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,Generic,1
111445,0.0,1.0,12.0,0.0,0.0,6.0,7.0,5.0,5.0,4.0,...,tcp,3454.559695,115.159023,69.747889,74.994711,0.203372,0.102021,0.101351,Analysis,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,...,tcp,6282.365879,240.295547,82.730666,159.926422,0.143100,0.101635,0.041465,DoS,1
103694,0.0,NaN,0.0,NaN,0.0,2.0,13.0,3.0,2.0,1.0,...,tcp,6986.891881,182.607109,102.387556,126.500711,0.135252,0.063701,0.071551,Reconnaissance,1
131932,0.0,2.0,0.0,0.0,0.0,16.0,16.0,NaN,16.0,16.0,...,udp,0.000000,0.000000,0.002000,0.000000,0.000000,0.000000,0.000000,Generic,1
146867,NaN,2.0,0.0,0.0,0.0,11.0,11.0,11.0,11.0,11.0,...,udp,0.000000,0.000000,0.003000,0.000000,0.000000,0.000000,0.000000,Generic,1


In [23]:
val_set

,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,...,proto,sjit,djit,sinpkt,dinpkt,tcprtt,synack,ackdat,attack_cat,label
15482,0.0,2.0,0.0,0.0,0.0,NaN,19.0,18.0,19.0,18.0,...,udp,0.000000,0.000000,0.002000,0.000000,0.000000,0.000000,0.000000,Generic,1
133349,0.0,2.0,0.0,0.0,0.0,16.0,16.0,17.0,16.0,NaN,...,udp,0.000000,0.000000,0.008000,0.000000,0.000000,0.000000,0.000000,Generic,1
80485,0.0,2.0,0.0,0.0,0.0,6.0,6.0,2.0,3.0,2.0,...,unas,0.000000,0.000000,0.009000,NaN,0.000000,0.000000,0.000000,Exploits,1
29972,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,1.0,...,sctp,0.000000,0.000000,0.005000,0.000000,NaN,0.000000,0.000000,DoS,1
18339,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,4.0,2.0,...,udp,0.000000,0.000000,0.009000,0.007000,0.000000,NaN,NaN,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94427,0.0,2.0,0.0,0.0,NaN,9.0,9.0,5.0,5.0,3.0,...,any,NaN,0.000000,0.009000,0.000000,0.000000,0.000000,0.000000,Reconnaissance,1
2244,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,1.0,...,tcp,4740.846082,107.475352,75.947444,66.683109,0.135008,0.082334,0.052674,Exploits,1
17130,0.0,0.0,0.0,0.0,0.0,4.0,2.0,4.0,5.0,2.0,...,udp,0.000000,0.000000,0.010000,0.004000,0.000000,0.000000,0.000000,Normal,0
14327,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,tcp,37634.196170,44066.712030,430.115950,502.054155,0.327325,0.209178,0.118147,Exploits,1


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [24]:
class FeatureImputer(BaseEstimator, TransformerMixin):
  def __init__(self, num_strategy="constant", num_fill_value=0, cat_strategy="constant", cat_fill_value="missing", encode_attack_cat=True):
      self.num_strategy = num_strategy
      self.num_fill_value = num_fill_value
      self.cat_strategy = cat_strategy
      self.cat_fill_value = cat_fill_value
      self.encode_attack_cat = encode_attack_cat
      self.imputer = None
      self.label_encoder = None

  def fit(self, X, y=None):
      self.columns_to_drop = ['id', 'label']
      X = X.drop(columns=self.columns_to_drop, errors='ignore')

      self.imputer = ColumnTransformer(
          transformers=[
              ("num", SimpleImputer(strategy=self.num_strategy, fill_value=self.num_fill_value), NUMERICALFEATURES),
              ("cat", SimpleImputer(strategy=self.cat_strategy, fill_value=self.cat_fill_value), CATEGORICALFEATURES),
          ],
          remainder="passthrough"
      )

      self.imputer.fit(X)

      if self.encode_attack_cat and "attack_cat" in X.columns:
          self.label_encoder = LabelEncoder()
          self.label_encoder.fit(X["attack_cat"])
          print("Labels in LabelEncoder:", self.label_encoder.classes_)

      return self

  def transform(self, X):
      X = X.drop(columns=self.columns_to_drop, errors='ignore')

      X_transformed = pd.DataFrame(self.imputer.transform(X), columns=self.imputer.get_feature_names_out())

      X_transformed.columns = [col.split("__")[-1] for col in X_transformed.columns]

      if self.encode_attack_cat and "attack_cat" in X.columns:
          X_transformed["attack_cat"] = self.label_encoder.transform(X["attack_cat"])

      return X_transformed

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [25]:
initial_count = len(train_set)
df_no_duplicates = train_set.drop_duplicates()
dropped_count = initial_count - len(df_no_duplicates)

print(f"Number of dropped rows: {dropped_count}")

Number of dropped rows: 0


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [26]:
class FeatureCreator(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
      self.fitted_cols = X.columns
      return self

  def transform(self, X):
      # Create new columns
      X['total_bytes'] = X['sbytes'] + X['dbytes']
      # X['packet_rate'] = X['total_bytes'] / (X['dur'] + 1e-9)
      X['byte_ratio'] = X['sbytes'] / (X['dbytes'] + 1e-9)
      X['src_to_dst_ratio'] = X['ct_srv_src'] / (X['ct_srv_dst'] + 1e-9)
      # X['port_usage_ratio'] = X['ct_dst_sport_ltm'] / (X['ct_src_dport_ltm'] + 1e-9)

      return X

class FeatureDropper(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
      return self

  def transform(self, X):
      return X.drop(columns=CAT_FEAT)

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [27]:
class FeatureScaler(BaseEstimator, TransformerMixin):
  def __init__(self):
      self.scaler = None
      self.numerical_cols = None

  def fit(self, X, y=None):
      self.numerical_cols = NUMERICALFEATURES
      self.scaler = StandardScaler().fit(X[self.numerical_cols])
      return self

  def transform(self, X):
      X_scaled = X.copy()
      X_scaled[self.numerical_cols] = self.scaler.transform(X[self.numerical_cols])

      return X_scaled

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [ ]:
# Write your code here

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [ ]:
# Write your code here

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

In [ ]:
# Write your code here

### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [28]:
def merge_features(X_train, X_test, n_clusters=5, random_state=None):
    data = pd.concat([X_train, X_test], axis=0, ignore_index=True)
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init='auto')
    data['cluster'] = kmeans.fit_predict(data)
    X_train_clustered, X_test_clustered = train_test_split(data, test_size=len(X_test), random_state=random_state, shuffle=False)
    return X_train_clustered, X_test_clustered

class pca_modified(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=2):
        self.pca = None
        self.num_features = None
        self.n_components = n_components

    def fit(self, X, y=None):
        self.num_features = NUMERICALFEATURES
        self.pca = PCA(n_components=self.n_components).fit(X[self.num_features])
        return self

    def transform(self, X):
        X_pca = X.copy()
        principal_components = self.pca.transform(X[self.num_features])

        for i in range(self.n_components):
            X_pca[f'PC{i + 1}'] = principal_components[:, i]

        X_pca = X_pca.drop(columns=[col for col in self.num_features if col in X_pca.columns])

        return X_pca

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [29]:
pipeline = Pipeline([
    ("imputer", FeatureImputer(num_strategy="constant", num_fill_value=0, cat_strategy="constant", cat_fill_value="missing")),  # Handle missing values
    ("featurecreator", FeatureCreator()),
    ("featurescaler", FeatureScaler()),
    ("dropper", FeatureDropper()),
    ("pca", pca_modified(n_components=5))
])

In [30]:
train_set = pipeline.fit_transform(train_set, train_set[TARGET])
val_set = pipeline.transform(val_set)

Labels in LabelEncoder: ['Analysis' 'Backdoor' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal'
 'Reconnaissance' 'Shellcode' 'Worms']


# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [31]:
class KNN:
  def __init__(self, k=5, metric='minkowski', p=2):
      self.k = k
      self.metric = metric

      if self.metric == 'manhattan':
          self.p = 1
      elif self.metric == 'euclidean':
          self.p = 2
      else:
          self.p = p

  def _get_nearest_neighbours(self, test):
      distances = np.linalg.norm(self.X_train - test, ord=self.p, axis=1)
      indices = np.argsort(distances)[:self.k]

      return indices

  def fit(self, X_train, y_train):
      if isinstance(X_train, pd.DataFrame):
          if X_train.columns.empty:
              self.X_train = X_train.values.astype(float)
          else:
              self.X_train = X_train.iloc[:, :-1].values.astype(float)
      else:
          self.X_train = X_train.astype(float)

      self.y_train = y_train

  def _predict_instance(self, row):
      indices = self._get_nearest_neighbours(row)
      labels = [self.y_train.iloc[neighbour] for neighbour in indices]

      prediction = max(set(labels), key=labels.count)

      return prediction

  def predict(self, X_test):
      if isinstance(X_test, pd.DataFrame):
          if X_test.columns.empty:
              X_test = X_test.values.astype(float)
          else:
              X_test = X_test.iloc[:, :-1].values.astype(float)
      else:
          X_test = X_test.astype(float)

      start_time = time.time()

      with concurrent.futures.ProcessPoolExecutor() as executor:
          results = list(tqdm(executor.map(self._predict_instance, X_test), total=len(X_test)))

      elapsed_time = time.time() - start_time

      print(f"Prediction completed in {elapsed_time:.2f} seconds.")

      return np.array(results)

  def save(self, path):
      pickle.dump(self, open(path, 'wb'))

  @staticmethod
  def load(path):
      return pickle.load(open(path, 'rb'))

## B. Naive Bayes

In [34]:
from collections import defaultdict

class NaiveBayes:
    def __init__(self):
        self.class_probs = None
        self.mean = None
        self.variance = None

    def fit(self, X, y):
        self.class_probs = self._calc_class_prob(y)
        self.mean, self.variance = self._calc_mean_var(X, y)

    def _calc_class_prob(self, y):
        count_cl = defaultdict(int)
        total = len(y)
        for label in y:
            count_cl[label] += 1
        class_probs = {label: count / total for label, count in count_cl.items()}
        return class_probs

    def _calc_mean_var(self, X, y):
        unique = np.unique(y)
        mean = {}
        variance = {}
        for label in unique:
            data = X[y == label]
            mean[label] = np.mean(data, axis=0)
            variance[label] = np.var(data, axis=0)
        return mean, variance

    def _gauss(self, x, mean, variance):
        exp = np.exp(-((x - mean) ** 2) / (2 * variance))
        return (1 / (np.sqrt(2 * np.pi * variance))) * exp

    def _calc_feature_probs(self, features, label):
        class_prob = np.log(self.class_probs[label])

        for i, feature in enumerate(features):
            mean = self.mean[label][i]
            variance = self.variance[label][i]
            epsilon = 1e-10
            if variance < epsilon:
                variance = epsilon

            class_prob += np.log(
                self._gauss(feature, mean, variance)
            )

        return class_prob

    def predict(self, X):
        results = []
        for sample in X.values:
            class_probs = {
                label: self._calc_feature_probs(sample, label)
                for label in self.class_probs
            }
            predicted_class = max(class_probs, key=class_probs.get)
            results.append(predicted_class)

        return np.array(results)

    def save(self, path):
        pickle.dump(self, open(path, 'wb'))

    @staticmethod
    def load(path):
        return pickle.load(open(path, 'rb'))

## C. ID3

In [35]:
class ID3DecisionTree:
  def __init__(self):
      self.tree = None
      self.default_class = None

  def entropy(self, y):
      class_counts = Counter(y)
      total = len(y)
      return -sum(
          (count / total) * np.log2(count / total)
          for count in class_counts.values()
          if count > 0
      )

  def information_gain(self, X_column, y):
      total_entropy = self.entropy(y)
      values, counts = np.unique(X_column, return_counts=True)
      weighted_entropy = sum(
          (counts[i] / len(y)) * self.entropy(y[X_column == value])
          for i, value in enumerate(values)
      )
      return total_entropy - weighted_entropy

  def best_split(self, X, y):
      best_gain = -1
      best_feature = None
      for feature in range(X.shape[1]):
          gain = self.information_gain(X[:, feature], y)
          if gain > best_gain:
              best_gain = gain
              best_feature = feature
      return best_feature

  def build_tree(self, X, y, features):
      if len(set(y)) == 1:
          return y[0]
      if len(features) == 0 or X.shape[1] == 0:
          return Counter(y).most_common(1)[0][0]

      best_feature = self.best_split(X, y)
      tree = {features[best_feature]: {}}

      feature_values = np.unique(X[:, best_feature])
      for value in feature_values:
          subset_indices = X[:, best_feature] == value
          subset_X = X[subset_indices]
          subset_y = y[subset_indices]
          new_features = np.delete(features, best_feature)
          new_X = np.delete(subset_X, best_feature, axis=1)
          tree[features[best_feature]][value] = self.build_tree(
              new_X, subset_y, new_features
          )
      return tree

  def fit(self, X, y):
      X = np.array(X)
      y = np.array(y)
      self.default_class = Counter(y).most_common(1)[0][0]
      features = np.array([f"Feature {i}" for i in range(X.shape[1])])
      self.tree = self.build_tree(X, y, features)

  def predict_one(self, x, tree):
      if not isinstance(tree, dict):
          return tree
      root_feature = next(iter(tree))
      feature_idx = int(root_feature.split()[-1])
      if feature_idx >= len(x):
          return self.default_class
      feature_value = x[feature_idx]
      subtree = tree[root_feature].get(feature_value, self.default_class)
      return self.predict_one(x, subtree)

  def predict(self, X):
      X = np.array(X)
      return np.array([self.predict_one(x, self.tree) for x in X])

  def save(self, path):
      pickle.dump(self, open(path, 'wb'))

  @staticmethod
  def load(path):
      return pickle.load(open(path, 'rb'))

## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [37]:
def match(train, test):
  train_feat = train.columns.tolist()
  test_feat = test.columns.tolist()

  for col in test_feat:
      if col not in train_feat:
          test = test.drop(col, axis=1)

  for col in train_feat:
      if col not in test_feat:
          test[col] = 0

  test = test[train_feat]

  return test

val_set = match(train_set, val_set)

In [38]:
knn = KNN(k=10)
nb = NaiveBayes()
id3 = ID3DecisionTree()

In [39]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

knn_import = KNeighborsClassifier(n_neighbors=10)
nb_import = GaussianNB()
id3_import = DecisionTreeClassifier(criterion='entropy', random_state=42)

In [40]:
X_train = train_set.drop(TARGET, axis=1)
y_train = train_set[TARGET]

X_val = val_set.drop(TARGET, axis=1)
y_val = val_set[TARGET]

X_train, X_val = merge_features(X_train, X_val, n_clusters=9, random_state=RANDOM_STATE)

In [49]:
knn.fit(X_train, y_train)
val_sample_indices = np.random.choice(len(X_val), size=int(0.1 * len(X_val)), replace=False)
X_val_sample = X_val.iloc[val_sample_indices]
y_val_sample = y_val.iloc[val_sample_indices]

y_val_pred = knn.predict(X_val_sample)

f1 = f1_score(y_val_sample, y_val_pred, average='macro')
print(f"F1 score for {knn.__class__.__name__} on 10% of validation set: {f1:.4f}")

100%|██████████| 3506/3506 [02:26<00:00, 23.96it/s]


Prediction completed in 146.73 seconds.
F1 score for KNN on 10% of validation set: 0.6079


In [48]:
knn_import.fit(X_train, y_train)
val_sample_indices = np.random.choice(len(X_val), size=int(0.1 * len(X_val)), replace=False)
X_val_sample = X_val.iloc[val_sample_indices]
y_val_sample = y_val.iloc[val_sample_indices]

y_val_pred = knn_import.predict(X_val_sample)

f1 = f1_score(y_val_sample, y_val_pred, average='macro')
print(f"F1 score for {knn_import.__class__.__name__} on 10% of validation set: {f1:.4f}")

F1 score for KNeighborsClassifier on 10% of validation set: 0.5912


In [50]:
nb.fit(X_train, y_train)
val_sample_indices = np.random.choice(len(X_val), size=int(0.1 * len(X_val)), replace=False)
X_val_sample = X_val.iloc[val_sample_indices]
y_val_sample = y_val.iloc[val_sample_indices]

y_val_pred = nb.predict(X_val_sample)

f1 = f1_score(y_val_sample, y_val_pred, average='macro')
print(f"F1 score for {nb.__class__.__name__} on 10% of validation set: {f1:.4f}")

<ipython-input-34-3425b0832d4d>:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean = self.mean[label][i]
<ipython-input-34-3425b0832d4d>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  variance = self.variance[label][i]
<ipython-input-34-3425b0832d4d>:45: RuntimeWarning: divide by zero encountered in log
  class_prob += np.log(


F1 score for NaiveBayes on 10% of validation set: 0.0951


In [51]:
nb_import.fit(X_train, y_train)
val_sample_indices = np.random.choice(len(X_val), size=int(0.1 * len(X_val)), replace=False)
X_val_sample = X_val.iloc[val_sample_indices]
y_val_sample = y_val.iloc[val_sample_indices]

y_val_pred = nb_import.predict(X_val_sample)

f1 = f1_score(y_val_sample, y_val_pred, average='macro')
print(f"F1 score for {nb_import.__class__.__name__} on 10% of validation set: {f1:.4f}")

F1 score for GaussianNB on 10% of validation set: 0.0552


In [ ]:
id3.fit(X_train, y_train)
val_sample_indices = np.random.choice(len(X_val), size=int(0.1 * len(X_val)), replace=False)
X_val_sample = X_val.iloc[val_sample_indices]
y_val_sample = y_val.iloc[val_sample_indices]

y_val_pred = id3.predict(X_val_sample)

f1 = f1_score(y_val_sample, y_val_pred, average='macro')
print(f"F1 score for {id3.__class__.__name__} on 10% of validation set: {f1:.4f}")

F1 score for ID3DecisionTree on 10% of validation set: 0.2114


In [52]:
id3_import.fit(X_train, y_train)
val_sample_indices = np.random.choice(len(X_val), size=int(0.1 * len(X_val)), replace=False)
x_val_sample = X_val.iloc[val_sample_indices]
y_val_sample = y_val.iloc[val_sample_indices]

y_val_pred = id3_import.predict(X_val_sample)

f1 = f1_score(y_val_sample, y_val_pred, average='macro')
print(f"F1 score for {id3_import.__class__.__name__} on 10% of validation set: {f1:.4f}")

F1 score for DecisionTreeClassifier on 10% of validation set: 0.1056


## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [41]:
train = train_data
test = test_data

In [42]:
test_id = test["id"]
test.drop(columns=["id"], inplace=True)

In [43]:
class FeatureImputerTest(BaseEstimator, TransformerMixin):
  def __init__(self, num_strategy="constant", num_fill_value=0, cat_strategy="constant", cat_fill_value="missing"):
      self.num_strategy = num_strategy
      self.num_fill_value = num_fill_value
      self.cat_strategy = cat_strategy
      self.cat_fill_value = cat_fill_value
      self.imputer = None

  def fit(self, X, y=None):
      self.num_cols = NUMERICALFEATURES
      self.cat_cols = CATEGORICALFEATURES
      self.imputer = ColumnTransformer(
          transformers=[
              ("num", SimpleImputer(strategy=self.num_strategy, fill_value=self.num_fill_value), self.num_cols),
              ("cat", SimpleImputer(strategy=self.cat_strategy, fill_value=self.cat_fill_value), self.cat_cols),
          ]
      )

      self.imputer.fit(X)
      return self

  def transform(self, X):
      X_transformed = pd.DataFrame(self.imputer.transform(X), columns=self.num_cols + self.cat_cols)
      return X_transformed

In [44]:
pipeline2 = Pipeline([
    ("imputer", FeatureImputerTest(num_strategy="constant", num_fill_value=0, cat_strategy="constant", cat_fill_value="missing")),  # Handle missing values
    ("featurecreator", FeatureCreator()),
    ("featurescaler", FeatureScaler()),
    ("dropper", FeatureDropper()),
    ("pca", pca_modified(n_components=5))
])

In [45]:
pipeline2.fit(train, train[TARGET])
train = pipeline.fit_transform(train, train[TARGET])
test = pipeline2.transform(test)

Labels in LabelEncoder: ['Analysis' 'Backdoor' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal'
 'Reconnaissance' 'Shellcode' 'Worms']


In [46]:
test = match(train, test)
test.drop(columns={TARGET}, inplace=True)

<ipython-input-46-4305ce1aaf51>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(columns={TARGET}, inplace=True)


In [47]:
X_train = train.drop([TARGET], axis=1)
y_train = train[TARGET]

X_test = test
X_train, X_test = merge_features(X_train, X_test, n_clusters=9, random_state=RANDOM_STATE)

nb = NaiveBayes()
knn = KNN(k=10)
id3 = ID3DecisionTree()

# nb.fit(X_train, y_train)
knn.fit(X_train, y_train)
# id3.fit(X_train, y_train)

# y_pred_nb = nb.predict(X_test)
y_pred_knn = knn.predict(X_test)
# y_pred_id3 = id3.predict(X_test)

df_submission = pd.DataFrame(data={'id': test_id, 'attack_cat': y_pred_knn})
df_submission

df_submission.to_csv("submission.csv", index=False)

# Validation Predictions
y_val_pred = knn.predict(X_val)

# Compute F1 Score on Validation Set
f1_val = f1_score(y_val, y_val_pred, average='macro')
print(f"F1 Score on Validation Set: {f1_val:.4f}")

100%|██████████| 20583/20583 [14:57<00:00, 22.93it/s]


Prediction completed in 900.95 seconds.


100%|██████████| 35069/35069 [25:08<00:00, 23.25it/s]


Prediction completed in 1513.31 seconds.
F1 Score on Validation Set: 0.6076


# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

KNN is the best-performing algorithm among the three, both in scratch implementation and using a library, because KNN is well-suited for complex data with clear distance patterns. Naive Bayes has the lowest performance since its assumption of feature independence does not align with the complexity of the UNSW-NB15 dataset. ID3 performs better in the scratch implementation compared to the library implementation, demonstrating the effectiveness of manual implementation in handling decision trees with more explicit control.

It is better to impute the missing data instead of dropping, because dropping the rows will result in the training data to be much smaller in size and will decrease the accuracy.

Feature scaling does help in improving model performance due to some models being sensitive with data distribution.

In [56]:
# nb.fit(X_train, y_train)
# id3.fit(X_train, y_train)

knn.save("knn.pkl")
nb.save("naive_bayes.pkl")
id3.save("id3.pkl")

In [57]:
knn = KNN.load("knn.pkl")
nb = NaiveBayes.load("naive_bayes.pkl")
id3 = ID3DecisionTree.load("id3.pkl")